In [1]:
import pandas as pd
import numpy as np
import os
from rdkit import Chem 
from rdkit import RDLogger  
from rdkit.Chem import PandasTools, SaltRemover
import cirpy

data = pd.DataFrame()

# Data import
Here i import the data from the data forms they were taken as and format them similar to each other

## Hansen
Hansen dataset taken from: https://pubs.acs.org/doi/10.1021/ci900161g
Comprising 6512 molecules from CCRIS, Helma et al., Kazius et al., Feng et al., VITIC, and GeneTox databases
Given is the SMILES, CAS No., and Ames mutagenicty
Postivite Ames mutagenicity is defined as having a positive Ames result for any salmonella strain for either with or without S9. Based on this it is said to be more accurate as this result will not change with additional testing.

In [2]:
path_to_hansen = 'data/raw/raw_data/smiles_cas_N6512.smi'
hansen_data = pd.read_csv(path_to_hansen,sep='\t', names=["smiles","CASRN", "Ames"])
# path_to_hansen = 'data/raw/raw_data/Hansen_et_al_2009.csv'
# hansen_data = pd.read_csv(path_to_hansen, names=["smiles","CASRN", "Ames"])
data =  pd.concat([
            data,
            hansen_data
        ])

## Bruschweiler
Brushweiler dataset taken from: https://www.sciencedirect.com/science/article/pii/S0273230017301812 
Dataset made up of 23 azo dye cleavage products. Given is the CAS No, SMILES, and Ames results for S9 +/- for strain TA98 and then TA100

In [3]:
path_to_bruschweiler = "data/raw/raw_data/Bruschweiler2017_Ames_Source.csv"
f = pd.read_csv(path_to_bruschweiler).drop(['TA98-S9','TA98+S9','TA100-S9','TA100+S9'],axis=1); f.rename(columns={'SMILES':'smiles','Overall':'Ames'},inplace=True)
data =  pd.concat([data,f[["smiles",'CASRN',"Ames"]]])

## OECD data
taken from the OECD toolbox using the databases available.
Note that these files were loaded with excel and saved to change the encoding to help with python reading the files

In [4]:
def converter(value):
    if str(value) in ['Positive']:
        return 1
    elif value in ['Negative']:
        return 0
    else:
        return np.nan

## Adding OECD data
directory = 'data/raw/raw_data/OECD_inv_data/reencoded'
for filename in os.listdir(directory):
    if filename.endswith(".csv"): 
        try:
            df = pd.read_csv(os.path.join(directory, filename), sep='\t', encoding='utf16',low_memory=False)
            if set(['SMILES','CAS Number']).issubset(df.columns) and any(col in df.columns for col in ['OVERALL','Value.MeanValue']):
                pass
            else:
                df = pd.read_csv(os.path.join(directory, filename), encoding='utf16',low_memory=False) 
                if set(['SMILES','CAS Number']).issubset(df.columns) and any(col in df.columns for col in ['OVERALL','Value.MeanValue']):  
                    pass
                else:
                    print(filename,"couldn't be loaded please check")
                    continue
            df["smiles"] = df["SMILES"]
            df['CASRN'] = df['CAS Number']
            if "OVERALL" in df.columns:
                df = df[pd.notna(df['OVERALL']) & pd.notna(df['SMILES'])]
                df["Ames"] = df['OVERALL'].apply(lambda x: converter(x))

                if len(df[pd.notna(df['Ames'])]) == 0:
                    df = df[pd.notna(df['Value.MeanValue']) & pd.notna(df['SMILES'])]
                    df["Ames"] = df['Value.MeanValue'].apply(lambda x: converter(x))

            else:
                df = df[pd.notna(df['Value.MeanValue']) & pd.notna(df['SMILES'])]
                df["Ames"] = df['Value.MeanValue'].apply(lambda x: converter(x))
            df = df[pd.notna(df['Ames'])]
            df["Ames"] = df["Ames"].astype(int)
            data = pd.concat([data,df[["smiles", "Ames"]]])
            print(filename, "DONE")

        except:
            print(filename,"couldn't be loaded please check")

Canada_DSL.csv DONE
COSING.csv DONE
DSSTOX.csv DONE
ECHA_PR.csv DONE
EINECS.csv DONE
HPVC_OECD.csv DONE
METI_Japan.csv DONE
NICNAS.csv DONE
REACH_ECB.csv DONE
TSCA.csv DONE
US_HPV_Challenge.csv DONE


# Data manipulation

In [5]:
def format_data(df):    
    def check_Ames(df):
        df = df[df['Ames'].isin([1,0])]
        return df

    def find_can_smi(df):
        def try_can_smi(x):
            try:
                return Chem.MolToSmiles(Chem.MolFromSmiles(x))
            except:
                
                # print('Molecule lost:',x)
                return "not found"
        df["smiles"] = df["smiles"].apply(lambda x: try_can_smi(x))
        df = df.drop_duplicates(subset=['smiles'])
        df = df[df['smiles'] != "not found"]
        return df
    def clean_molecules(df):
        def remove_salts(smiles):
            if not smiles == np.nan:
                remover = SaltRemover.SaltRemover()
                mol = Chem.MolFromSmiles(smiles)
                if not mol or not smiles:
                    print(smiles)
                    return np.nan
                
                res = remover.StripMol(mol)
                if Chem.MolToSmiles(res):
                    return Chem.MolToSmiles(res)
                else:
                    # print("Molecule salt removal removed whole molecule", smiles, "Molecule was retained but please check")
                    return smiles
            else:
                print(smiles)
                return np.nan
        # def remove_solutions(smiles):
            ####
        df['smiles'] = df['smiles'].apply(lambda x: remove_salts(x))
        # df['smiles'] = df['smiles'].apply(lambda x: remove_solutions(x))
        return df
    export_df = df.copy()
    export_df = check_Ames(export_df)
    export_df = find_can_smi(export_df)
    export_df = clean_molecules(export_df)
    return export_df

def only_aromatic_amines(df):
    def get_mol(row):
        try:
            mol = Chem.MolFromSmiles(row['smiles'])
            test = Chem.MolToSmiles(mol)
            return mol
        except:
            try:
                mol = Chem.MolFromSmiles(pcp.get_compounds(row['CASRN'], 'name')[0].canonical_smiles)
                test = Chem.MolToSmiles(mol)
                return mol
            except:
                print("error on:",row['smiles'],row['CASRN'])
                return "Not found"
    def AA_check(row):
        m = get_mol(row)
        if m != "Not found":
            for a in m.GetAtoms():
                if a.GetAtomicNum() == 7 and any([n.GetIsAromatic() for n in a.GetNeighbors()]) and not a.IsInRing()  and (a.GetFormalCharge() >= 0) and len([n.GetIsAromatic() for n in a.GetNeighbors()]) ==1:     ## Checks for a nitrogen and then if the neighbors of the nitrogen are aromatic and also checks that it isn't cationic
                    return 1
                else: 
                    return 0
    export_df = df.copy()
    export_df["is AA"] = export_df.apply(lambda row: AA_check(row),axis=1)
    export_df = export_df[export_df["is AA"] == 1]
    return export_df

## Hansen all molecules

In [6]:
# I disable error logs as i account for them generally in my get mol function by searching for a better smiles using the CAS number. Please use for debugging if needed
RDLogger.DisableLog('rdApp.*') 
data_final = format_data(hansen_data)
a = data_final
print("original total:", len(hansen_data))
print('final total:', len(data_final))
print("original Ames split:", "yes="+str(len(hansen_data[hansen_data["Ames"] == 1])), "no="+str(len(hansen_data[hansen_data["Ames"] == 0])))
print("final Ames split:", "yes="+str(len(data_final[data_final["Ames"] == 1])), "no="+str(len(data_final[data_final["Ames"] == 0])))
data_final = data_final[["smiles",'Ames']].reset_index(drop = True)
b = data_final
##### saving data in two forms
    # csv form
data_final.to_csv('data/raw/hansen_raw/Hansen_all_mols.csv', index=False)
    # sdf form
PandasTools.AddMoleculeColumnToFrame(data_final,'smiles','Molecule')
data_final['idx'] = data_final.index
Chem.PandasTools.WriteSDF(data_final, "data/raw/hansen_raw/Hansen_all_mols.sdf", molColName='Molecule', properties=list(data_final.columns), idName = "idx")

original total: 6512
final total: 6505
original Ames split: yes=3503 no=3009
final Ames split: yes=3496 no=3009


## All data Aromatic amines

In [7]:
# I disable error logs as i account for them generally in my get mol function by searching for a better smiles using the CAS number. Please use for debugging if needed
RDLogger.DisableLog('rdApp.*') 
data_final = format_data(data)
data_final = only_aromatic_amines(data_final)

print("original total:", len(data))
print('final total:', len(data_final))
print("original Ames split:", "yes="+str(len(data[data["Ames"] == 1])), "no="+str(len(data[data["Ames"] == 0])))
print("final Ames split:", "yes="+str(len(data_final[data_final["Ames"] == 1])), "no="+str(len(data_final[data_final["Ames"] == 0])))
data_final = data_final[["smiles",'Ames']].reset_index(drop = True)

##### saving data in two forms
    # csv form
data_final.to_csv("data/raw/selected_molecules.csv", index=False)
    # sdf form
PandasTools.AddMoleculeColumnToFrame(data_final,'smiles','Molecule')
data_final['idx'] = data_final.index
Chem.PandasTools.WriteSDF(data_final, "data/raw/selected_molecules.sdf", molColName='Molecule', properties=list(data_final.columns), idName = "idx")

original total: 146079
final total: 457
original Ames split: yes=38580 no=107499
final Ames split: yes=306 no=151
